# Step-by-step run of alphazero self-play & training.


In [17]:
import os
import time
from pathlib import Path
from collections import defaultdict, Counter
import asyncio

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e"
os.makedirs(DATA_DIR, exist_ok=True)

MODEL_DIR = Path.cwd().parent / "models" / "rgizero-e2e"
os.makedirs(MODEL_DIR, exist_ok=True)

%load_ext line_profiler

✅ Imports successful
Using device: mps
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [18]:
DEBUG_MODE = True     # Set options to make debugger work properly. Single worker, etc.
LOAD_MODEL = False
TRAIN_MODEL = True
MODEL_SIZE = "small"  # "tiny" or "small" or"large" or "xl"
NUM_SIMULATIONS = 50

# If False, we still load previous games from disk.
PLAY_GAMES = True
PLAY_GAMES_V2 = True
NUM_GAMES = 50
MAX_TRAINING_ITERS = 100
SAVE_FILE_NAME = f'train_{NUM_GAMES}'

## Step 1: Set up history-wrapped game


In [19]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
all_actions = game.all_actions()
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(all_actions))
n_max_context = max_game_length + 2

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {base_game.__class__.__name__}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

✅ Using HistoryTrackingGame from module
Game: Connect4Game, Players: 2, Actions: [1, 2, 3, 4, 5, 6, 7]


## Step 2: Create or load model.


In [20]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer, ActionHistoryTransformerEvaluator
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}


def create_random_model(config: TransformerConfig, action_vocab_size, num_players,  seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed) # Ensure numpy operations are also seeded
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    model = ActionHistoryTransformer(config=config, action_vocab_size=action_vocab_size, num_players=num_players)
    model.to(device)
    return model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]

if LOAD_MODEL:
    raise NotImplementedError("Model loading not implemented")
else:
    model = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

# Step 3: Play games to generate training data


In [21]:
from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async
from tqdm.asyncio import tqdm

async def play_games_async(num_games: int, player_factory):
    tasks = []
    async def create_player_and_create_game():
        t0 = time.time()
        player = player_factory()
        game_result = await play_game_async(game, [player, player])
        t1 = time.time()
        game_result['time'] = t1 - t0
        return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results

async def play_generation_async(model, num_games, simulations=NUM_SIMULATIONS):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=1024, verbose=False)

    master_rng = np.random.default_rng(42)
    async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=simulations)

    await async_evaluator.start()
    results = await play_games_async(num_games=num_games, player_factory=async_player_factory)
    await async_evaluator.stop()
    return results

def print_game_stats(results):
    print("Winner Stats:")
    winner_stats = Counter(result['winner'] for result in results)
    print(f"Winner counts: win[1]={100*winner_stats[1]/sum(winner_stats.values()):.2f}% win[2]={100*winner_stats[2]/sum(winner_stats.values()):.2f}%, n={sum(winner_stats.values())}")
    game_lengths = [len(result['action_history']) for result in results]
    print(f"Game Length min: {min(game_lengths)}, max: {max(game_lengths)}, mean: {np.mean(game_lengths):.2f}")
    print("Winner Stats by initial move:")
    dd = defaultdict(Counter)
    for result in results:
        dd[result['action_history'][0]][result['winner']] += 1
    for action, counts in sorted(dd.items()):
        print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}")


In [22]:
if PLAY_GAMES:
    print(f"Playing {NUM_GAMES} games, simulations={NUM_SIMULATIONS}, model_size={MODEL_SIZE}")
    results_gen0 = await play_generation_async(model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS)

Playing 50 games, simulations=50, model_size=small


100%|██████████| 50/50 [00:04<00:00, 12.35it/s]


In [23]:
if PLAY_GAMES:
    print_game_stats(results_gen0)

## Resunts from 1000 connect4 games with random model.
# Winner by initial move:
#   a=1: n=121 win[1]=59.50% counts=Counter({1: 72, 2: 49})
#   a=2: n=139 win[1]=58.99% counts=Counter({1: 82, 2: 57})
#   a=3: n= 95 win[1]=63.16% counts=Counter({1: 60, 2: 35})
#   a=4: n=160 win[1]=78.75% counts=Counter({1: 126, 2: 34})
#   a=5: n=146 win[1]=65.07% counts=Counter({1: 95, 2: 51})
#   a=6: n=176 win[1]=61.36% counts=Counter({1: 108, 2: 68})
#   a=7: n=163 win[1]=53.37% counts=Counter({1: 87, 2: 76})

Winner Stats:
Winner counts: win[1]=52.00% win[2]=48.00%, n=50
Game Length min: 7, max: 30, mean: 14.52
Winner Stats by initial move:
  a=1: n=  3 win[1]=33.33% counts=Counter({2: 2, 1: 1})
  a=2: n=  7 win[1]=57.14% counts=Counter({1: 4, 2: 3})
  a=3: n=  5 win[1]=60.00% counts=Counter({1: 3, 2: 2})
  a=4: n= 10 win[1]=60.00% counts=Counter({1: 6, 2: 4})
  a=5: n=  7 win[1]=57.14% counts=Counter({1: 4, 2: 3})
  a=6: n= 11 win[1]=36.36% counts=Counter({2: 7, 1: 4})
  a=7: n=  7 win[1]=57.14% counts=Counter({1: 4, 2: 3})


## Step 3: Confirm we can read & write to trajectory_dataset


In [24]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset, build_trajectory_loader
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

def write_trajectory_dataset(results, action_vocab, save_file_name):
    td_builder = TrajectoryDatasetBuilder(action_vocab)
    for game_result in results:
        add_trajectory(game_result, action_vocab, td_builder)

    td_builder.save(DATA_DIR, save_file_name)    

if PLAY_GAMES:
    write_trajectory_dataset(results_gen0, action_vocab, SAVE_FILE_NAME)


## Load dataset


In [25]:
# Load dataset
num_workers = 0 if DEBUG_MODE else 4

td = TrajectoryDataset(DATA_DIR, SAVE_FILE_NAME, block_size=n_max_context)
trajectory_loader = build_trajectory_loader(
    DATA_DIR, SAVE_FILE_NAME, block_size=n_max_context, batch_size=1,
    device=device, workers=num_workers)

## Train model


In [26]:
from rgi.rgizero.train import Trainer, TrainConfig

train_config = TrainConfig(
    model_name="connect4-e2e",
    model_version="v1",

    eval_interval = 250,  # keep frequent because we'll overfit
    eval_iters = 200,
    log_interval = 10_000,  # don't print too too often

    # we expect to overfit on this small dataset, so only save when val improves
    always_save_checkpoint = False,

    gradient_accumulation_steps = 1,
    batch_size = 64,

    learning_rate = 1e-3,  # with baby networks can afford to go a bit higher
    max_iters = MAX_TRAINING_ITERS,
    lr_decay_iters = 5000,  # make equal to max_iters usually
    min_lr = 1e-4,  # learning_rate / 10 usually
    beta2 = 0.99,  # make a bit bigger because number of tokens per iter is small

    warmup_iters = 100,  # not super necessary potentially
)

trainer = Trainer(
    model=model,
    train_config=train_config,
    train_loader=trajectory_loader,
    val_loader=trajectory_loader,  # TODO: Create separate validation loader
    device=device
)

if TRAIN_MODEL:
    trainer.train()

num decayed parameter tensors: 19, with 50,880 parameters
num non-decayed parameter tensors: 11, with 298 parameters
using fused AdamW: False
Training epoch 0 of 10
step 0: train loss 0.2272, val loss 0.2272
iter 0: loss 1.0209, time 184.82ms
Training epoch 1 of 10
Training epoch 2 of 10


In [27]:
import dataclasses
# Save model
MODEL_PATH = MODEL_DIR / f"{SAVE_FILE_NAME}-round1.pt"

checkpoint = {
    'model': model.state_dict(),
    'model_config': dataclasses.asdict(model.config),
    'vocab': action_vocab.to_dict(),
    'iter_num': trainer.iter_num,
    'best_val_loss': trainer.best_val_loss,
    'num_players': game.num_players(state_0),
}
torch.save(checkpoint, DATA_DIR / f"{SAVE_FILE_NAME}-round1-checkpoint.pt")

loaded_checkpoint = torch.load(DATA_DIR / f"{SAVE_FILE_NAME}-round1-checkpoint.pt")
loaded_model = ActionHistoryTransformer(
    config=TransformerConfig(**loaded_checkpoint['model_config']),
    action_vocab_size=loaded_checkpoint['vocab']['vocab_size'],
    num_players=loaded_checkpoint['num_players']
)
loaded_model.to(device)


ActionHistoryTransformer(
  (action_embedding): Embedding(8, 32)
  (transformer): Transformer(
    (wpe): Embedding(44, 32)
    (dropout): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=32, out_features=96, bias=False)
          (c_proj): Linear(in_features=32, out_features=32, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=32, out_features=128, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=128, out_features=32, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
  )
  (ln_f): LayerNorm()
  (policy_value_head): PolicyValueHead(
    (policy): Linear(in_features=32, out_features=8, bias=True)
    (valu

In [28]:
reload_local_modules(verbose=False)


serial_evaluator_v2 = ActionHistoryTransformerEvaluator(loaded_model, device=device, block_size=block_size, vocab=action_vocab)
async_evaluator_v2 = AsyncNetworkEvaluator(base_evaluator=serial_evaluator_v2, max_batch_size=1024, verbose=False)

master_rng = np.random.default_rng(42)
async_evaluator_factory_v2 = lambda: AsyncNetworkEvaluator(base_evaluator=serial_evaluator_v2, max_batch_size=1024, verbose=False)
async_player_factory_v2 = lambda: AlphazeroPlayer(game, async_evaluator_v2, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=NUM_SIMULATIONS)

if PLAY_GAMES_V2:
    await async_evaluator_v2.start()
    results_v2 = asyncio.run(play_games_async(num_games=NUM_GAMES, player_factory=async_player_factory_v2))
    await async_evaluator_v2.stop()

100%|██████████| 50/50 [00:04<00:00, 11.93it/s]


In [30]:
results_v2[0]['legal_policies'][0]


array([0.18, 0.1 , 0.16, 0.12, 0.18, 0.08, 0.18], dtype=float32)

In [ ]:
reload_local_modules(verbose=False)

results_v2_100 = asyncio.run(play_generation_async(loaded_model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS))
print_game_stats(results_v2_100)

100%|██████████| 50/50 [00:03<00:00, 12.57it/s]

Winner Stats:
Winner counts: win[1]=44.00% win[2]=56.00%, n=50
Game Length min: 8, max: 29, mean: 15.36
Winner Stats by initial move:
  a=1: n=  5 win[1]=60.00% counts=Counter({1: 3, 2: 2})
  a=2: n=  5 win[1]=60.00% counts=Counter({1: 3, 2: 2})
  a=3: n=  9 win[1]=22.22% counts=Counter({2: 7, 1: 2})
  a=4: n=  6 win[1]=50.00% counts=Counter({1: 3, 2: 3})
  a=5: n= 14 win[1]=21.43% counts=Counter({2: 11, 1: 3})
  a=6: n=  4 win[1]=75.00% counts=Counter({1: 3, 2: 1})
  a=7: n=  7 win[1]=71.43% counts=Counter({1: 5, 2: 2})
